In [ ]:
# KNN - Shreeya
 
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.utils import to_categorical
 
# Initialize the scaler and the KNN classifier
scaler = StandardScaler()
knn = KNeighborsClassifier()
 
# Create a pipeline that first scales the data then applies KNN
pipeline = Pipeline([('scaler', scaler), ('knn', knn)])
 
# Parameters of the KNN classifier to tune
param_grid = {
    'knn__n_neighbors': [3, 5, 7, 9, 11],  # Example range; can be expanded based on computational resources
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}
 
# Grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=1)
grid_search.fit(train_features, train_labels)
 
# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
 
# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
test_pred = best_model.predict(test_features)
print("Test set accuracy: ", accuracy_score(test_labels, test_pred))
print(classification_report(test_labels, test_pred))
print(confusion_matrix(test_labels, test_pred))

# GRU Model - Shreeya
 
num_features_per_axis = 6   #6 features (e.g., mean, min, max, std, kurtosis, skew for each axis)
num_axes = 3    # x, y, z axes
timesteps = 8  # Time steps per gesture
 
# Reshape the feature data
X = extracted_features.drop('gesture', axis=1).values
X = X.reshape(-1, timesteps, num_features_per_axis * num_axes)
 
# Encode labels
encoder = LabelEncoder()
y = encoder.fit_transform(df['gesture'])
y = to_categorical(y)  # Convert labels to one-hot encoding
 
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
 
#Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
 
model = Sequential([
    GRU(128, return_sequences=True, input_shape=(timesteps, num_features_per_axis * num_axes)),
    Dropout(0.2),
    GRU(64),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')  # Output layer: number of gesture types
])
 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
 
#training the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)
 
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)
 